In [8]:
import pandas as pd
import sqlalchemy as sal

engine = sal.create_engine('mssql+pyodbc://siva\\SQLEXPRESS/namaste_sql_assignements?driver=ODBC+Driver+17+for+SQL+Server')
conn=engine.connect()


In [16]:
def extract():
    df_product=pd.read_csv("C:\\Users\\skesi\\Downloads\\product.txt",sep=',')
    df_products_db=pd.read_sql_query('select * from products' , conn)
    return df_product,df_products_db
def transfrom(df_product,df_products_db):
    df_merged=pd.merge(df_product,df_products_db, how='left',on='product_id')
    df_insert=df_merged[df_merged['price_y'].isna()]
    df_insert_final=df_insert.iloc[:,0:3]
    df_insert_final.columns=df_product.columns
    
    df_update=df_merged[df_merged['price_y'].notna()]
    df_update_final=df_insert.iloc[:,0:3]
    df_update_final.columns=df_product.columns
    return df_insert_final, df_update_final

def load_staging(df_update_final):
    df_update_final.to_sql('products_stg',con=conn , index=False , if_exists = 'replace')
    conn.commit()

def updates():
    query = sal.text("update products set price = products_stg.price, product_name=products_stg.product_name from products_stg where products.product_id=products_stg.product_id")
    p = conn.execute(query)
    conn.commit()

def insert(df_insert_final):
    df_insert_final.to_sql('products',conn,index=False,if_exists='append')

In [18]:
#extract
df_product,df_products_db=extract()

#transfrom
df_insert_final,df_update_final=transfrom(df_product,df_products_db)

#inserting new rows
insert(df_insert_final)

#loading changes to staging
load_staging(df_update_final)

#updating change in main table
updates()